In [1]:
import pyarrow as pa    
import pyarrow.parquet as pq
import numpy as np
import re
from pythainlp import util
import os
from tqdm import tqdm
from datasets import Dataset

In [2]:
dataset_path = os.path.abspath("datasets")
clean_dataset_path = os.path.abspath("datasets/clean")

dataset_paths = os.listdir(dataset_path)
dataset_paths.remove('clean')
dataset_paths = [os.path.join(dataset_path, dir) for dir in dataset_paths]
dataset_names = [os.path.basename(path).split('.')[0] for path in dataset_paths]

In [3]:
# read datasets from path
datasets = dict(zip(dataset_names,[pq.read_table(path) for path in dataset_paths]))

In [4]:
t = pa.table({
    "text":['UNK hlello`` \'\' world','\n\n\n-lrb- olo  \n\n(SHRF) ain\'t ‘yolo”\n'],
    "sum":['UNKhelUNKlo','olo']
}, schema=pa.schema([
    ('text', pa.string()),
    ('sum', pa.string())
]))
t.schema

text: string
sum: string

### tools function

In [5]:
def batch_processing(function, iterable, num_batch=65536):
    def batch(iterable, n=num_batch):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx:min(ndx + n, l)]

    if num_batch == 0:
        return list(map(function, iterable))
    else:
        out_bat = []
        for bat in batch(iterable, num_batch):
            out_bat = out_bat + list(map(function, bat))
        return out_bat
    

def pa_map(func):
    batch_size = 32768
    def wrapper(table):
        dict = table.to_pydict()
        schema = table.schema
        n_sample = table.num_rows
        del table
        # dict['text'] = list(map(func, dict['text'])) 
        # dict['sum'] = list(map(func, dict['sum']))
        dict['text'] = batch_processing(func, dict['text'], batch_size)
        dict['sum'] = batch_processing(func, dict['sum'], batch_size)
        table = pa.table(dict, schema=schema)
        del dict
        return table
    return wrapper

### normalize spatial token in each datasets

In [6]:
@pa_map
def remove_spcial_tokens_gigaword(input):
    input = str(input)
    input = input.replace("-lrb-", "")
    input = input.replace("UNK", "")
    return input

In [7]:
@pa_map
def remove_spcial_tokens_thaisum(input):
    input = str(input)
    input = input.replace("(SHRF)","")
    input = input.replace("(SSA/SSPP)","")
    input = input.replace("(SSPP)","")
    input = input.replace("(SSA)","")
    input = input.replace(u"\xa0"," ")
    input = re.sub(r"\[.*\]","", input)
    return input

### normalize text

In [8]:
@pa_map
def normalize_space(input):
    input = str(input)
    input = input.strip()
    input = re.sub("\n+", "\n", input)
    input = re.sub(" +", " ", input)
    input = re.sub("\t+", "\t", input)
    return input


@pa_map
def normalize_quotation(input):
    input = str(input)
    input = re.sub("“", "\"", input)
    input = re.sub("”", "\"", input)
    input = re.sub("‘", "'", input)
    input = re.sub("’", "'", input)
    input = re.sub("‘", "'", input)
    input = re.sub("’", "'", input)
    return input    


@pa_map
def expand_contractions(input):
    
    input = str(input)
    
    def replace(match):
        return contractions_dict[match.group(0)]
    
    contractions_dict = { "ain't": "are not", "'s":" is", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "‘cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "that'd": "that would", "that'd've": "that would have", "there'd": "there would", "there'd've": "there would have", "they'd": "they would", "they'd've": "they would have","they'll": "they will",
    "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not","what'll": "what will", "what'll've": "what will have", "what're": "what are", "what've": "what have", "when've": "when have", "where'd": "where did", "where've": "where have",
    "who'll": "who will", "who'll've": "who will have", "who've": "who have", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"
    }

    contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
    return contractions_re.sub(replace, input)

### normalize thai text datasets

In [9]:
@pa_map
def vowel_normalize(input: str):
    '''
    input: list of string of string 
    output: list of string or string
    normalize เ เ, ํ า, double of same vowel(such as โโ, แแ,  ูู, ุุ ,่่) , and wrong order of vowel and tonal(such as ก่ิง)
    '''
    return util.normalize(str(input))


@pa_map
def thai_digit_to_arabic_digit(input: str):
    '''
    input: list of string of string 
    output: list of string or string
    change thai digit(๑๒๓๔) to string of arabic digit
    
    example:
        thai_digit_to_arabic_digit('๑๒๓๔กขคabc') -> '1234กขคabc'
    '''
    return util.thai_digit_to_arabic_digit(str(input))

### use functions

In [10]:
func_q = [
    remove_spcial_tokens_gigaword,
    remove_spcial_tokens_thaisum,
    normalize_space,
    normalize_quotation,
    expand_contractions
]

th_func_q = [
    vowel_normalize,
    thai_digit_to_arabic_digit
]

thai_dataset = [
    'thaisum'
]
  
  
for key in datasets.keys():
    for i in tqdm(range(len(func_q)), desc=f'process normalize datasets {key}.'):
        datasets[key] = func_q[i](datasets[key])


for key in thai_dataset:
    for i in tqdm(range(len(th_func_q)), desc=f'process normalize datasets {key}.'):
        datasets[key] = func_q[i](datasets[key])

process normalize datasets thaisum.: 100%|██████████| 2/2 [00:52<00:00, 26.17s/it]


In [11]:
# def dataset_pro(func):
#     def wrap(input):
#         return{
#             "text": func(input['text']),
#             "sum": func(input['sum'])
#         }
#     return wrap

### Save cleaned datasets

In [12]:
for i in tqdm(range(len(dataset_names)), desc="Save cleaned dataset"):
    dataset_name = dataset_names[i]
    file_name = dataset_name + ".parquet"
    path = os.path.join(clean_dataset_path, file_name)
    pq.write_table(table= datasets[dataset_name], where=path)

Save cleaned dataset:   0%|          | 0/4 [00:00<?, ?it/s]

Save cleaned dataset: 100%|██████████| 4/4 [01:42<00:00, 25.51s/it]
